In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.legend_handler import HandlerLine2D
from math import log, ceil
from cycler import cycler

fig, ax = plt.subplots()
fig.set_size_inches(4, 3)
AI = [0.125,0.25,0.5,1,2,4,8,16,32,64,128,256,512]
axes = plt.gca()
axes.set_ylim([1e-2,.3e1])
ax.set_xscale('log')
ax.set_yscale('log')

node=[45,32,22,16,22,8];
f_rd=[1,1.09,2.38,3.21,4.17,3.85];
v_rd=[1,0.93,0.84,0.75,0.68,0.62];
c_rd=[1,0.7,0.33,0.21,0.13,0.08];
p_rd=[1,0.66,0.54,0.38,0.25,0.12];
p_con=[1,0.71,0.52,0.39,0.29,0.22];

fact=[1e-10]*6;
# Tf=[0.1877,0.0830,0.0367,0.0163,0.0072,0.0032];
# Tm=[0.3906,0.2461,0.1550,0.0977,0.0615,0.0388];

Tf = 1/(53.28*1e9);
Tm = 1/(25.6*1e9);
Ef = 670 * 1e-12;
Em = 795 * 1e-12;
p0 = 122;
Ef=np.multiply(Ef,p_con);
Em=np.multiply(Em,p_con);

labels = ['8 nm','11 nm','16 nm','22 nm','32 nm','45 nm'];
Emodel = [None]*13;
ax.set_prop_cycle('color', ['navy', 'darkcyan', 'mediumaquamarine', 'yellowgreen','orange', 'orangered'])

for i in range(5,-1,-1):
    p0 = 122*p_con[i]
    E0 = p0*Tf;
    Bt = [Tm/Tf];
    Be = Em[i]/Ef[i];
    Ef1 = Ef[i] + E0;
    eta = Ef[i]/Ef1;
    Be1 = eta * Be + (1-eta) * np.maximum(0,np.subtract(Bt,AI));
    E = Ef1*(1+Be1/AI);
    Emodel = 1/E/1e9;
    line1, = ax.plot(AI, Emodel,label='%s' % labels[5-i]) 
    
p0 = 122*p_con[5];
E0 = p0*Tf;
Bt = Tm/Tf;
Ef1 = Ef[5] + E0;
eta = Ef[5]/Ef1;
Be = Em[5]/Ef[5];
Be1 = eta * Be + (1-eta) * np.maximum(0,np.subtract(Bt,AI));
E = Ef1*(1+Be1/AI);
Emodel = 1/E/1e9;

vline_min=0.01;
y_min=0.012;
x_value=4.8773;
y_value = np.interp(x_value, AI, Emodel);
plt.vlines(x=x_value, ymin=vline_min, ymax=y_value, color='black', zorder=2, linestyle=':')
plt.text(x_value-1.2,y_min,'FFT',rotation=90, fontsize=7,verticalalignment='bottom')

x_value=211.3476;
y_value = np.interp(x_value, AI, Emodel);
plt.vlines(x=x_value, ymin=vline_min, ymax=y_value, color='black', zorder=2, linestyle=':')
plt.text(x_value-50,y_min,'FMM P2P',rotation=90, fontsize=7,verticalalignment='bottom')

x_value=23.5898;
y_value = np.interp(x_value, AI, Emodel);
plt.vlines(x=x_value, ymin=vline_min, ymax=y_value, color='black', zorder=2, linestyle=':')
plt.text(x_value-6,y_min,'FMM M2L (Cartesian)',rotation=90, fontsize=7,verticalalignment='bottom')

x_value=11.8024;
y_value = np.interp(x_value, AI, Emodel);
plt.vlines(x=x_value, ymin=vline_min, ymax=y_value, color='black', zorder=2, linestyle=':')
plt.text(x_value-3,y_min,'FMM M2L (EC+FFT)',rotation=90, fontsize=7,verticalalignment='bottom')

x_value=0.1849;
y_value = np.interp(x_value, AI, Emodel);
plt.vlines(x=x_value, ymin=vline_min, ymax=y_value, color='black', zorder=2, linestyle=':')
plt.text(x_value-0.05,y_min,'SpMV',rotation=90, fontsize=7,verticalalignment='bottom')

x_value=0.3549;
y_value = np.interp(x_value, AI, Emodel);
plt.vlines(x=x_value, ymin=vline_min, ymax=y_value, color='black', zorder=2, linestyle=':')
plt.text(x_value-0.09,y_min,'Stencil',rotation=90, fontsize=7,verticalalignment='bottom')

sns.set_context(rc={"font.size":6})   
plt.legend(handler_map={line1: HandlerLine2D(numpoints=4)})
ax.set_xticklabels(['1/8','1/4','1/2','1','2','4','8','16','32','64','128','256','512'], fontsize=6)
ax.tick_params(axis='both', which='major', labelsize=6)
ax.set_xticks(AI)                                                       
ax.set_ylabel('Performance (GFLOP/J)',size=8)
ax.set_xlabel('Arithmetic Intensity (FLOP/Byte)',size=8)

legend = plt.legend(fontsize=8,loc=2,frameon=True)
frame = legend.get_frame()
frame.set_facecolor('white')
sns.set_style("whitegrid")
ax.xaxis.grid(False)
ax.yaxis.grid(True)
plt.show()
fig = ax.get_figure()
fig.savefig('powProject.pdf',bbox_inches='tight')